In [ ]:
import numpy as np

In [ ]:
%run experiments/concept-emergence2/notebooks/gmm/generate_data.py
%run experiments/concept-emergence2/notebooks/gmm/plot_data.py
%run experiments/concept-emergence2/notebooks/gmm/igmn.py

In [ ]:
def predict(igmn, n_samples, min, max):
    x = np.linspace(min, max, n_samples)
    y = [igmn.likelihood([x_i]) for x_i in x]
    return x.reshape((-1,)), y

In [ ]:
# # TODO: the scale matters a lot, if the scale is too small (between 0 and 1, the predictions do not make sense)
# X, Y = gen_data(k=3, dim=2, points_per_cluster=100, lim=[0, 100], spread=[1, 5])
# # shuffle X and Y (but keep the correspondence)
# X, Y = shuffle_data(X, Y)
# # plot the data
# plot_data(X, y=Y)

In [ ]:
# TODO: the scale matters a lot, if the scale is too small (between 0 and 1, the predictions do not make sense)
X, Y = gen_data(k=3, dim=1, points_per_cluster=100, lim=[0, 100], spread=[1, 5])
# shuffle X and Y (but keep the correspondence)
X, Y = shuffle_data(X, Y)
# plot the data
plot_data(X, y=Y)

In [ ]:
parameters = {'beta': 0.1, 'all_adults_criterion': True, 'age_min': 10, 'acc_min': 9, 'max_components': 20}

# igmn = IGMN(input_dim=1, beta=0.1, all_adults_criterion=False, age_min=5.0, acc_min=3.0, max_components=20, rank_type = "diag", closest_n=20
igmn = IGMN(input_dim=2, **parameters, rank_type = "diag", closest_n=20)
num_epochs = 1
for epoch in range(num_epochs):
    igmn.fit(X)

log_probs = igmn.score_samples(X)
score = np.exp(log_probs).sum()


#print("Number of components: ", len(igmn.components))
#print("Components centered at: ", [comp['mu'].item() for comp in igmn.components])
#print("Training fit: ", score)
#x_min, x_max = min(X)-1, max(X)+1
# n_samples = 10000
# x, y = predict(igmn, n_samples, x_min, x_max)

#print("IGMN has ", len(igmn.components), " components")
#print("GMM integral: ", scipy.integrate.simpson(y, x), " or ", scipy.integrate.quad(lambda x: igmn.likelihood(x), -50, 150))
#plot_gmm(X, Y, x_min, x_max, x, y)

In [ ]:
import optuna

def objective(trial):
    beta = trial.suggest_float('beta', 0.0000000001, 0.1)
    all_adults_criterion = trial.suggest_categorical('all_adults_criterion', [True, False])
    age_min = trial.suggest_int('age_min', 2.0, 10.0)
    acc_min = trial.suggest_int('acc_min', 2.0, 10.0)
    max_components = trial.suggest_int('max_components', 1, 30)
    igmn = IGMN(input_dim=2, beta=beta, all_adults_criterion=all_adults_criterion, age_min=age_min, acc_min=acc_min, max_components=max_components, rank_type = "diag", closest_n=20)
    num_epochs = 1
    for _ in range(num_epochs):
        igmn.fit(X)
    # x, y = predict(igmn, n_samples, x_min, x_max)
    log_probs = igmn.score_samples(X)
    score = np.exp(log_probs).sum()
    return score
    

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)  # number of iterations

In [ ]:
study.best_params


### GMM

In [ ]:
from sklearn.mixture import GaussianMixture

def predict2(gmm, n_samples, x_min, x_max):
    x = np.linspace(x_min, x_max, n_samples)
    y = [np.exp(gmm.score_samples([x_i])) for x_i in x]
    return x, y

In [ ]:
gmm = GaussianMixture(n_components=20, covariance_type="diag")
gmm.fit(X);

In [ ]:
log_probs = gmm.score_samples(X)
log_probs

In [ ]:
np.exp(log_probs)

In [ ]:
print(np.exp(log_probs).sum())
print(np.exp(log_probs).sum())
print(sum(np.exp(gmm.score_samples([x_i])) for x_i in X))

In [ ]:
x, y = predict2(gmm, n_samples, x_min, x_max)

plot_gmm(X, Y, x_min, x_max, x, y)

In [ ]:
y